# ЛР2 (Линейные модели)

In [220]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

# Предобработка данных для задачи классификации

In [221]:
data_classification = pd.read_csv(r'C:\Users\xwxsz\Desktop\Video_games_esrb_rating.csv')

label_encoder = LabelEncoder()
data_classification['title'] = label_encoder.fit_transform(data_classification['title'])
data_classification['esrb_rating'] = label_encoder.fit_transform(data_classification['esrb_rating'])

X = data_classification.drop('esrb_rating', axis=1) 
y = data_classification['esrb_rating'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Классификация (бейзлайн)

In [222]:
# Создание и обучение модели логистической регрессии

model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [223]:
# Предикт

y_pred = model.predict(X_test)

In [224]:
# Оценка

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')  # Если много классов, используем 'weighted'

In [225]:
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.8680738786279684
F1 Score: 0.8703572874280974


# Классификация (улучшенный бейзлайн)

In [226]:
# Масштабирование данных и логистическая регрессия в пайплайне
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Масштабирование
    ('logreg', LogisticRegression(random_state=42, max_iter=1000))  # Логистическая регрессия
])

In [227]:
# Настройка параметров для GridSearchCV
param_grid = {
    'logreg__C': [0.01, 0.1, 1, 10, 100],  # Параметр регуляризации
    'logreg__penalty': ['l1', 'l2'],  # Тип регуляризации
    'logreg__solver': ['liblinear', 'saga'],  # Решатели для поддержки L1/L2
    'logreg__class_weight': [None, 'balanced']  # Учет дисбаланса классов
}

In [228]:
# Используем GridSearchCV для поиска оптимальных параметров
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, 
                           scoring='f1_weighted', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('logreg',
                                        LogisticRegression(max_iter=1000,
                                                           random_state=42))]),
             n_jobs=-1,
             param_grid={'logreg__C': [0.01, 0.1, 1, 10, 100],
                         'logreg__class_weight': [None, 'balanced'],
                         'logreg__penalty': ['l1', 'l2'],
                         'logreg__solver': ['liblinear', 'saga']},
             scoring='f1_weighted', verbose=1)

In [229]:
# Лучшая модель
best_model = grid_search.best_estimator_

In [230]:
# Прредикт
y_pred = best_model.predict(X_test)

In [231]:
# Оценка 
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

In [232]:
print(f"Accuracy (улучшенный бейзлайн): {accuracy}")
print(f"F1 Score (улучшенный бейзлайн): {f1}")

Accuracy (улучшенный бейзлайн): 0.8733509234828496
F1 Score (улучшенный бейзлайн): 0.8754449315013263


# Классификация (cамостоятельная имплементация)

In [233]:
# Масштабирование данных для лучшей сходимости градиентного спуска
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Самостоятельная реализация логистической регрессии с многоклассовой поддержкой
class LogisticRegressionCustom:
    def __init__(self, learning_rate=0.001, n_iterations=1000, regularization='l2', reg_strength=0.1):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.regularization = regularization
        self.reg_strength = reg_strength

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))  # Защита от переполнений

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.weights = np.random.randn(X.shape[1], len(self.classes)) * 0.01  # Инициализация весов
        self.bias = np.zeros(len(self.classes))

        # Преобразуем y в матрицу для многоклассовой классификации
        y_matrix = np.zeros((y.size, len(self.classes)))
        for i, label in enumerate(self.classes):
            y_matrix[:, i] = (y == label).astype(int)

        for i in range(self.n_iterations):
            scores = np.dot(X, self.weights) + self.bias
            probs = self.softmax(scores)
            error = probs - y_matrix

            gradient_weights = np.dot(X.T, error) / y.size
            gradient_bias = np.sum(error, axis=0) / y.size

            if self.regularization == 'l2':
                gradient_weights += self.reg_strength * self.weights / y.size
            elif self.regularization == 'l1':
                gradient_weights += self.reg_strength * np.sign(self.weights) / y.size

            self.weights -= self.learning_rate * gradient_weights
            self.bias -= self.learning_rate * gradient_bias

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Для стабильности
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def predict(self, X):
        scores = np.dot(X, self.weights) + self.bias
        probs = self.softmax(scores)
        return np.argmax(probs, axis=1)

# Обучение модели
log_reg_custom = LogisticRegressionCustom(learning_rate=0.01, n_iterations=1000, regularization='l2', reg_strength=0.1)
log_reg_custom.fit(X_train, y_train)

# Предикт
y_pred_custom = log_reg_custom.predict(X_test)

# Оценка 
accuracy_custom = accuracy_score(y_test, y_pred_custom)
f1_custom = f1_score(y_test, y_pred_custom, average='weighted')

print(f"Accuracy (до улучшения): {accuracy_custom}")
print(f"F1 (до улучшения): {f1_custom}")

Accuracy (до улучшения): 0.8364116094986808
F1 (до улучшения): 0.8370614931868936


In [234]:
# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Создание полиномиальных признаков
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Балансировка классов с помощью SMOTE
smote = SMOTE(random_state=42)
X_train_poly_balanced, y_train_balanced = smote.fit_resample(X_train_poly, y_train)

# Оптимизация гиперпараметров с ручным перебором
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_iterations': [500, 1000, 2000]
}

best_params = None
best_f1 = 0
for learning_rate in param_grid['learning_rate']:
    for n_iterations in param_grid['n_iterations']:
        logreg_custom = LogisticRegressionCustom(learning_rate=learning_rate, n_iterations=n_iterations, regularization='l2', reg_strength=0.1)
        logreg_custom.fit(X_train_poly_balanced, y_train_balanced)
        y_pred_custom = logreg_custom.predict(X_test_poly)
        f1 = f1_score(y_test, y_pred_custom, average='weighted')
        if f1 > best_f1:
            best_f1 = f1
            best_params = {'learning_rate': learning_rate, 'n_iterations': n_iterations}

# Обучение с лучшими параметрами
logreg_custom_best = LogisticRegressionCustom(**best_params, regularization='l2', reg_strength=0.1)
logreg_custom_best.fit(X_train_poly_balanced, y_train_balanced)
y_pred_custom_best = logreg_custom_best.predict(X_test_poly)

# Оценка
accuracy_custom = accuracy_score(y_test, y_pred_custom_best)
f1_custom = f1_score(y_test, y_pred_custom_best, average='weighted')

print("Accuracy (после улучшения):", accuracy_custom)
print("F1-Score (после улучшения):", f1_custom)

Accuracy (после улучшения): 0.8759894459102903
F1-Score (после улучшения): 0.8765678642101511


# Предобработка данных для задачи регрессии

In [235]:
data_regression = pd.read_excel(r'C:\Users\xwxsz\Desktop\Real estate valuation data set.xlsx')
data_regression

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.916667,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.916667,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583333,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500000,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833333,5.0,390.56840,5,24.97937,121.54245,43.1
...,...,...,...,...,...,...,...,...
409,410,2013.000000,13.7,4082.01500,0,24.94155,121.50381,15.4
410,411,2012.666667,5.6,90.45606,9,24.97433,121.54310,50.0
411,412,2013.250000,18.8,390.96960,7,24.97923,121.53986,40.6
412,413,2013.000000,8.1,104.81010,5,24.96674,121.54067,52.5


In [236]:
X = data_regression.drop(columns=['Y house price of unit area'])
y = data_regression['Y house price of unit area']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Регрессия (бейзлайн)

In [237]:
# Масштабирование данных для улучшения сходимости градиентного спуска
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Создание и обучение модели линейной регрессии
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Предикт
y_pred = model.predict(X_test_scaled)

# Оценка 
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE (Бейзлайн): {mae}")
print(f"R² (Бейзлайн): {r2}")

MAE (Бейзлайн): 5.418032735899375
R² (Бейзлайн): 0.674541419569227


# Регрессия (улучшеный бейзлайн)

In [238]:
# Создание полиномиальных признаков
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Поиск гиперпараметров через GridSearchCV
param_grid = {
    'fit_intercept': [True, False],  # Решение, нужно ли вычислять свободный член
}

grid_search = GridSearchCV(LinearRegression(), param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train_poly, y_train)

# Лучшая модель после поиска по сетке
best_model = grid_search.best_estimator_

# Предикт
y_pred_best = best_model.predict(X_test_poly)

# Оценка
mae_best = mean_absolute_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

print(f"MAE (улучшенный бейзлайн): {mae_best}")
print(f"R² (улучшенный бейзлайн): {r2_best}")

MAE (улучшенный бейзлайн): 4.793900814579976
R² (улучшенный бейзлайн): 0.7496837454377854


# Регрессия (cамостоятельная имплементация)

In [239]:
class LinearRegressionCustom:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.theta = None
        self.bias = None

    def fit(self, X, y):
        m, n = X.shape
        self.theta = np.zeros(n)
        self.bias = 0

        # Градиентный спуск
        for _ in range(self.n_iterations):
            y_pred = self.predict(X)
            error = y_pred - y

            # Вычисление градиентов
            gradient_theta = (1/m) * np.dot(X.T, error)
            gradient_bias = (1/m) * np.sum(error)

            # Обновление параметров
            self.theta -= self.learning_rate * gradient_theta
            self.bias -= self.learning_rate * gradient_bias

    def predict(self, X):
        return np.dot(X, self.theta) + self.bias

    def score(self, X, y):
        y_pred = self.predict(X)
        return np.mean((y - y_pred) ** 2) 

In [240]:
# Обучение модели
model = LinearRegressionCustom(learning_rate=0.01, n_iterations=1000)
model.fit(X_train_scaled, y_train)

# Предикт
y_pred = model.predict(X_test_scaled)

# Оценка
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE (до улучшения): {mae}")
print(f"R² (до улучшения): {r2}")

MAE (до улучшения): 5.396581597717543
R² (до улучшения): 0.6772480363486442


In [241]:
class LinearRegressionWithRegularization:
    def __init__(self, learning_rate=0.0005, n_iterations=30000, lambda_reg=0.0001):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.lambda_reg = lambda_reg
        self.theta = None
        self.bias = None

    def fit(self, X, y):
        m, n = X.shape
        self.theta = np.zeros(n)
        self.bias = 0

        # Градиентный спуск с L2-регуляризацией
        for i in range(self.n_iterations):
            y_pred = self.predict(X)
            error = y_pred - y

            # Вычисление градиентов с учетом регуляризации
            gradient_theta = (1/m) * np.dot(X.T, error) + (self.lambda_reg / m) * self.theta
            gradient_bias = (1/m) * np.sum(error)

            # Обновление параметров
            self.theta -= self.learning_rate * gradient_theta
            self.bias -= self.learning_rate * gradient_bias

    def predict(self, X):
        return np.dot(X, self.theta) + self.bias

    def score(self, X, y):
        y_pred = self.predict(X)
        return mean_absolute_error(y, y_pred), r2_score(y, y_pred)

# Создаем полиномиальные признаки
poly = PolynomialFeatures(degree=2) 
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Обучение модели с полиномиальными признаками
model = LinearRegressionWithRegularization(learning_rate=0.0005, n_iterations=30000, lambda_reg=1e-05)
model.fit(X_train_poly, y_train)

# Оценка
mae, r2 = model.score(X_test_poly, y_test)
print(f"MAE (после улучшения): {mae}")
print(f"R² (после улучшения): {r2}")

MAE (после улучшения): 4.856235423013
R² (после улучшения): 0.7415093558756588
